# Summary

# Imports

In [1]:
import os
import shutil
import subprocess
import shlex
import tqdm
import yaml
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
pd.set_option("max_columns", 100)

In [3]:
%run spark.ipynb

# Parameters

In [4]:
NOTEBOOK_PATH = Path('validation_dataset')
NOTEBOOK_PATH

PosixPath('validation_dataset')

In [5]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH

PosixPath('/home/kimlab1/database_data/datapkg/adjacency-net-v2/notebooks/validation_dataset')

# `DATAPKG`

In [6]:
DATAPKG = {}

In [7]:
DATAPKG['uniparc-domain-wstructure'] = (
    Path(os.environ['DATAPKG_OUTPUT_DIR'])
    .joinpath("uniparc-domain-wstructure", "master")
)

# Load data

In [8]:
with NOTEBOOK_PATH.parent.joinpath("training_validation_test_split", "validation_domains.yaml").open("rt") as fin:
    validation_domains = yaml.load(fin)

# Copy training examples to a new path

In [9]:
adjacency_matrix_path = (
    DATAPKG['uniparc-domain-wstructure']
    .joinpath("remove_duplicate_matches", "adjacency_matrix.parquet")
)
adjacency_matrix_path

PosixPath('/home/kimlab1/database_data/datapkg_output_dir/uniparc-domain-wstructure/master/remove_duplicate_matches/adjacency_matrix.parquet')

In [10]:
output_adjacency_matrix_path = OUTPUT_PATH.joinpath("adjacency_matrix.parquet")
try:
    shutil.rmtree(output_adjacency_matrix_path)
except FileNotFoundError:
    pass
output_adjacency_matrix_path.mkdir(exist_ok=True)
output_adjacency_matrix_path

PosixPath('/home/kimlab1/database_data/datapkg/adjacency-net-v2/notebooks/validation_dataset/adjacency_matrix.parquet')

In [11]:
for domain in tqdm.tqdm_notebook(validation_domains, total=len(validation_domains)):
    out = (
        spark
        .read
        .parquet(adjacency_matrix_path.joinpath(domain).as_posix())
        .filter(F.col("pc_identity") >= 80)
        .coalesce(1)
        .write
        .save(path=output_adjacency_matrix_path.joinpath(domain).as_posix(), format='parquet', mode='overwrite')
    )
    assert out is None

HBox(children=(IntProgress(value=0, max=172), HTML(value='')))